**Phase 3: Implement Retrieval-Augmented Generation (RAG)**

In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma(
    collection_name="database",
    embedding_function=embedding_model,
    persist_directory="database"
)
vectorstore._collection
num_chunks = len(vectorstore.get()["ids"])
print(f"Number of chunks loaded from 'database': {num_chunks}")
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

C:\Users\krish\AppData\Local\Temp\ipykernel_38964\1599606810.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


C:\Users\krish\AppData\Local\Temp\ipykernel_38964\1599606810.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


Number of chunks loaded from 'database': 1731


In [4]:
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
llama = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)

Device set to use cuda:0


In [22]:
rag_prompt_template = PromptTemplate.from_template("""
You are a helpful assistant with deep chess knowledge.you are given a context and a question.
Use the following context to answer the user query:

Context:
{context}

User: {question}
Answer:
""")


In [14]:
def rag_chat(query: str):
    docs = retriever.get_relevant_documents(query)
    context = "\n\n".join([doc.page_content for doc in docs])
    prompt = rag_prompt_template.format(context=context, question=query)

    output = llama(prompt)[0]["generated_text"]
    
    if prompt in output:
        output = output.replace(prompt, "").strip()
    
    return output

In [27]:
user_query = "What is Castling?"
response = rag_chat(user_query)
print("💬", response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


💬 Castling is a move that allows you to protect your king and use your rook. Once all the squares between your rook and the king are free, you can move the king two square toward the square where the rook is while the rook moves to the square on the king’s other side. Look for an opportunity when your opponent disregards to castle, this is when you can launch an attack on the opponent’s king. This move is the only way where more than a single piece can be moved in a single move/turn.
